# Resumo do código cocó

### <u>Código que gera os ficheiros para estudo do comportamento da Delta</u>
---
O objectivo é receber dados da Delta e devolver um conjunto de métricas para prever roturas. Devolve um ficheiro que pode entrar no código 1 para juntar aos dados dos ninjas.

---
- Inputs

> __Dados completos da Delta em pastas de ficheiros__ (de azul a verde)
> - Stocks e trânsito, Sellout do dia anterior

                    ou

> __Ficheiro já completo__ (de vermelho a verde)
> - Stocks e trânsito, Sellout do dia anterior

- Outputs

> __Ficheiro com produtos em causa__ em formato Long

> __Métricas novas:__
> - Roturas de Stock e Pré-rotura
> - Sinal
> - Ciclos e Adequação de Stock
> - MSA (média de sellouts 10 dias antes)
> - STK (Stock disponível + trânsito)
> - (Novo) Balanço médio, mediano, liberal e conservador 
> - (Novo) Dias para a rotura de stock e de prateleira


In [4]:
%%time
import pandas as pd
import numpy as np
import datetime


def escrever_csv(dfa, nome): 
    dfa.to_csv(f'D:\\B&N Dados\\Delta\\Stocks\\StocksTotal\\{nome}.csv', index=False)

def escrever_txt(dfa, nome): 
    dfa.to_csv(f'D:\\B&N Dados\\Delta\\Stocks\\StocksTotal\\{nome}.txt', index=False, header=False)

def escrever_excel(dfa, nome):
    dfa.to_excel(f'D:\\B&N Dados\\Delta\\Stocks\\StocksTotal\\{nome}.xlsx' , index=False)

CPU times: total: 422 ms
Wall time: 766 ms


---

#  <span style="color:red"><u>Ler Ficheiro Completo</u> </span>

In [5]:
%%time

#Ler os ficheiros
df_2022 = pd.read_csv('D:\\B&N Dados\\Delta\\Stocks\\Stocks2022\\Stocks_Delta_2022_Limpo.csv')
df_2023 = pd.read_csv('D:\\B&N Dados\\Delta\\Diário\\Stocks_Delta_2023_Limpo.csv')

# Juntar as bases
dataframes = [df_2022, df_2023]
df_Fusão = pd.concat(dataframes, ignore_index=True)
df_Fusão['DATA']= pd.to_datetime(df_Fusão['DATA'], format='%Y-%m-%d')



# # Ficheiro de previsão
# df_Prophet = pd.read_csv('D:\\B&N Dados\\Delta\\Forecast\\Prophet.csv')
# df_Prophet['DATA']= pd.to_datetime(df_Prophet['DATA'], format='%Y-%m-%d')

# df_XGBoost = pd.read_csv('D:\\B&N Dados\\Delta\\Forecast\\XGBoost.csv')
# df_XGBoost['DATA']= pd.to_datetime(df_XGBoost['DATA'], format='%Y-%m-%d')

CPU times: total: 18.8 s
Wall time: 20.8 s


In [6]:
df_Fusão.head()

,DATA,EAN,DESC_ARTIGO,STORE,STORE_NAME,INTRANSIT,EXPECTED,PRES_STOCK,STOCK,STOCK_1_Dias_Antes,SELLOUT,SELLOUT_1_Dias_Antes,INTRANSIT_1_Dias_Antes,EXPECTED_1_Dias_Antes,PRES_STOCK_1_Dias_Antes,FLUXO
0,2022-03-23,5600692769695,100% SALICÓRNIA MOIDA 45G,1,CNT MATOSINHOS,0.0,0.0,10.0,23.0,23.0,0.0,0,NaN,NaN,NaN,NaN
1,2022-03-24,5600692769695,100% SALICÓRNIA MOIDA 45G,1,CNT MATOSINHOS,0.0,0.0,10.0,22.0,23.0,1.0,0,NaN,NaN,NaN,NaN
2,2022-03-25,5600692769695,100% SALICÓRNIA MOIDA 45G,1,CNT MATOSINHOS,0.0,0.0,10.0,22.0,22.0,0.0,1,NaN,NaN,NaN,NaN
3,2022-03-26,5600692769695,100% SALICÓRNIA MOIDA 45G,1,CNT MATOSINHOS,0.0,0.0,10.0,22.0,22.0,0.0,0,NaN,NaN,NaN,NaN
4,2022-03-27,5600692769695,100% SALICÓRNIA MOIDA 45G,1,CNT MATOSINHOS,0.0,0.0,10.0,22.0,22.0,0.0,0,NaN,NaN,NaN,NaN


In [7]:
df_Fusão[(df_Fusão.STORE_NAME=="CNT BRAGA N. ARCADA") & (df_Fusão.DESC_ARTIGO=="DOUBLE ESPRESSO GO CHILL DELTA 230ML") & (df_Fusão.DATA>"2023-01-03")].DATA.unique()

array(['2023-06-01T00:00:00.000000000', '2023-06-02T00:00:00.000000000',
       '2023-06-03T00:00:00.000000000', '2023-06-04T00:00:00.000000000',
       '2023-06-05T00:00:00.000000000', '2023-06-06T00:00:00.000000000',
       '2023-06-07T00:00:00.000000000', '2023-06-08T00:00:00.000000000',
       '2023-06-09T00:00:00.000000000', '2023-06-10T00:00:00.000000000',
       '2023-06-11T00:00:00.000000000', '2023-06-12T00:00:00.000000000',
       '2023-06-13T00:00:00.000000000', '2023-06-14T00:00:00.000000000',
       '2023-06-15T00:00:00.000000000', '2023-06-16T00:00:00.000000000',
       '2023-06-17T00:00:00.000000000', '2023-06-18T00:00:00.000000000',
       '2023-06-19T00:00:00.000000000', '2023-06-20T00:00:00.000000000',
       '2023-06-21T00:00:00.000000000', '2023-06-22T00:00:00.000000000',
       '2023-06-23T00:00:00.000000000', '2023-06-24T00:00:00.000000000',
       '2023-06-25T00:00:00.000000000', '2023-06-26T00:00:00.000000000',
       '2023-06-27T00:00:00.000000000', '2023-06-28

# <font color=red>Opcional:</font> Definir produtos em causa

- Produtos específicos

In [8]:
import clickhouse_connect
client = clickhouse_connect.get_client(host='ch.brandsandninjas.com', 
                                       port=443, 
                                       username='chninja', 
                                       password='ku43ueqnB5Q0AYb2C4FsJRTc7qX',
                                       database = "Delta")

#Ler o ficheiro

dfClick = client.query_df('SELECT * FROM GoChill4SemFotos2')


In [9]:
dfClick.head()

,DATA,STORE_NAME,FDS,Dia_da_Semana,DESC_ARTIGO,STORE_NAME2,Color,LatLong,Vendedor,STORE,Num_Produtos,Order_FDS,NinjaInfo,Rotura,Rotura_Consecutiva,Percentagem_de_Presenca,Order_Dia
0,2023-09-09,Continente Modelo Portalegre,9-10 Setembro,Sábado,CAFFELATTE GO CHILL DELTA 230 ML,Continente Modelo Portalegre,Produto Presente,"39.2776511 , -7.4317202",Andreia Gonçalves,218.0,2.0,1.0,1.0,0.0,0.0,0.4,6.0
1,2023-09-09,Continente Covilhã,9-10 Setembro,Sábado,CAFFELATTE GO CHILL DELTA 230 ML,Continente Covilhã,Produto Presente,"40.2713171 , -7.5024285",Bruno Gomes,212.0,4.0,1.0,1.0,0.0,0.0,0.8,6.0
2,2023-09-09,Continente Colombo,9-10 Setembro,Sábado,CAFFELATTE GO CHILL DELTA 230 ML,Continente Colombo,Produto em Rotura,"38.755174144937996 , -9.189612569714395",Brizida Almeida,9.0,1.0,1.0,0.0,1.0,0.0,0.2,6.0
3,2023-09-09,Continente Leiria,9-10 Setembro,Sábado,CAFFELATTE GO CHILL DELTA 230 ML,Continente Leiria,Produto Presente,"39.73288936955392 , -8.82407364242947",Filipa Dias,5.0,4.0,1.0,1.0,0.0,0.0,0.8,6.0
4,2023-09-09,Continente Modelo Portalegre,9-10 Setembro,Sábado,CAPPUCCINO GO CHILL DELTA 230 ML,Continente Modelo Portalegre,Produto Presente,"39.2776511 , -7.4317202",Andreia Gonçalves,218.0,2.0,1.0,1.0,0.0,0.0,0.4,6.0


> - Fazer **dfFinal**

In [10]:
produtos = dfClick.DESC_ARTIGO.unique()
lojas = dfClick.STORE_NAME.unique()
codlojas = dfClick.STORE.unique()

dfFinal = df_Fusão[(df_Fusão["DESC_ARTIGO"].isin(produtos)) & (df_Fusão["STORE"].isin(codlojas))].copy()
dfFinal = pd.merge(dfFinal, dfClick[["STORE","DATA","DESC_ARTIGO","NinjaInfo", "Percentagem_de_Presenca"]], how="outer", on = ["STORE","DATA","DESC_ARTIGO"])

In [11]:
dfVendedor=pd.read_excel("D:\\B&N Dados\\Delta\\Vendedor2.xlsx", sheet_name = "Lojas Sonae para o desafio")
dfVendedor = dfVendedor.rename(columns={"Cód. Loja":"STORE"})

# Criar coluna de reposição
dfFinal = pd.merge(dfFinal, dfVendedor[["STORE","Vendedor"]], how="left", on = "STORE")

---

# Colunas de métricas interessantes

> - ROTURA
> - PRÉ_ROTURA

In [12]:
# Definir coluna de rotura (se stock menor ou igual a 0 e existe Linear)

dfFinal["ROTURA"] = np.where((dfFinal["STOCK"] <= 0) & (dfFinal["PRES_STOCK"] > 0), 1, 0)
dfFinal["ROTURA_1_Dias_Antes"] = np.where((dfFinal["STOCK_1_Dias_Antes"] <= 0) & (dfFinal["PRES_STOCK_1_Dias_Antes"] > 0), 1, 0)


# Definir coluna de rotura (se stock menor ou igual a 0)

dfFinal["PRE_ROTURA"] = (dfFinal["STOCK"] < dfFinal["PRES_STOCK"]).astype(int)
dfFinal["PRE_ROTURA_1_Dias_Antes"] = (dfFinal["STOCK_1_Dias_Antes"] < dfFinal["PRES_STOCK_1_Dias_Antes"]).astype(int)

# Métricas 1, 4, 5 e 10 dias antes:

- INSTRANSIT
- EXPECTED
- SELLOUT
- CICLOS
- Dias para Rotura
- Adequação

In [13]:
# Quantos dias antes:

diasMet = [1, 4, 5, 10]

# Função para colunas de dias anteriores

def dias(df, dia, coluna):         #dia é quantos dias antes
    a=int(dia)

    valores = df.groupby(['DESC_ARTIGO', 'STORE'])[coluna].transform(lambda x: x.shift(a))
    valores[:a] = np.nan
    
    df.loc[:,f'{coluna}_{a}_Dias_Antes'] = valores

---

> - STOCK
> - SELLOUT
> - INTRANSIT
> - EXPECTED 
> - STK
> - FORNECIMENTO
> - CICLOS
> - Adequação

In [14]:
dfFinal["STK"] = dfFinal["STOCK"] + dfFinal["INTRANSIT"] + dfFinal["EXPECTED"]
dfFinal["FORNECIMENTO"] = dfFinal["INTRANSIT"] + dfFinal["EXPECTED"]
dfFinal["CICLOS"] = dfFinal["STOCK"]/dfFinal["PRES_STOCK"]
dfFinal["Adequacao"]= np.where(dfFinal["CICLOS"] > 1.1, "Stock Suficiente", 
                      np.where((dfFinal["CICLOS"] <= 1.1) & (dfFinal["INTRANSIT"]+dfFinal["EXPECTED"]+dfFinal["STOCK"]>=dfFinal["PRES_STOCK"]), "Stock Insuf c Forn Adequado", 
                      np.where((dfFinal["CICLOS"] <= 1.1) & (dfFinal["INTRANSIT"]+dfFinal["EXPECTED"]+dfFinal["STOCK"]<dfFinal["PRES_STOCK"]), "Stock Insuf c Forn Desadequado", 
                      "")))

for i in diasMet:   
    dias(dfFinal, i, "STOCK")
    dias(dfFinal, i, "SELLOUT")
    dias(dfFinal, i, "INTRANSIT")
    dias(dfFinal, i, "EXPECTED")
    dias(dfFinal, i, "STK")
    dias(dfFinal, i, "FORNECIMENTO")
    dias(dfFinal, i, "CICLOS")
    dias(dfFinal, i, "Adequacao")

> - MSA
>- Balance: sellout / soma stock disponível mais transito.

In [15]:
# MSA do dia = média dos sellouts dos 10 dias anteriores ao dia em causa

dfFinal["MSA10"] = dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_1_Dias_Antes'].transform(lambda x: x.rolling(window=10).mean())
dfFinal["MSA10Dp"] = dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_1_Dias_Antes'].transform(lambda x: x.rolling(window=10).std())

for i in diasMet:
    dias(dfFinal, i, "MSA10")
    

dfFinal["MSA20"] = dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_1_Dias_Antes'].transform(lambda x: x.rolling(window=20).mean())
dfFinal["MSA20Dp"] = dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_1_Dias_Antes'].transform(lambda x: x.rolling(window=20).std())
  
for i in diasMet:
    dias(dfFinal, i, "MSA20")


# Balance do dia = razão entre o sellout médio e o stock para o dia actual
dfFinal["Balance"] =  dfFinal["MSA10"] / dfFinal["STK"]

for i in diasMet:
    
    
    valores = i * dfFinal["Balance"].shift(i)
    valores[:i] = np.nan
    
    dfFinal.loc[:,'%s_%s_Dias_Antes' % ("Balance", i)] = valores
    
dfFinal.loc[dfFinal.STK == 0, "Balance"] = 2
    

# Colunas de métricas 30, 60, 120 e 180 dias antes

In [16]:
diasMetHist = [30, 60, 120, 180]

### Percentagens

>- Balance raw

In [17]:
dfFinal["Balance_Raw"] =  dfFinal["SELLOUT"] / dfFinal["STK"]

dfFinal["Balance_Raw_Count1"] = np.where(dfFinal["Balance_Raw"] < 0.5, 1, 0)

dfFinal["Balance_Raw_Count2"] = np.where(dfFinal["Balance_Raw"] < 0.8, 1, 0)

for i in diasMetHist:
    dfFinal[f"Percentagem_Balance_Raw_Count1_{i}"] = dfFinal.groupby(['DESC_ARTIGO', "STORE"])['Balance_Raw_Count1'].transform(lambda x: x.rolling(window=i, min_periods=1).mean())
    dfFinal[f"Percentagem_Balance_Raw_Count2_{i}"] = dfFinal.groupby(['DESC_ARTIGO', "STORE"])['Balance_Raw_Count2'].transform(lambda x: x.rolling(window=i, min_periods=1).mean())

> - Efeito fim de semana

In [18]:
#dfFinal['SELLOUT_fds'] = dfFinal[dfFinal['DATA'].dt.weekday.isin([4,5,6])]["SELLOUT"].copy()
#dfFinal['SELLOUT_semana'] = dfFinal[dfFinal['DATA'].dt.weekday.isin([0,1,2,3])]["SELLOUT"].copy()
dfFinal['SELLOUT_fds_1_Dias_Antes'] = dfFinal[dfFinal['DATA'].dt.weekday.isin([5,6,0])]["SELLOUT_1_Dias_Antes"].copy()
dfFinal['SELLOUT_semana_1_Dias_Antes'] = dfFinal[dfFinal['DATA'].dt.weekday.isin([1,2,3,4])]["SELLOUT_1_Dias_Antes"].copy()

dfFinal['SELLOUT_fds_Medio'] = dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_fds_1_Dias_Antes']\
    .transform(lambda x: x.rolling(window=30, min_periods=1).mean())
dfFinal['SELLOUT_semana_Medio'] = dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_semana_1_Dias_Antes']\
    .transform(lambda x: x.rolling(window=30, min_periods=1).mean())


for i in diasMetHist:
    dfFinal[f"Percentagem_Efeito_Fds_{i}"] = ((dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_fds_1_Dias_Antes']\
                                               .transform(lambda x: x.rolling(window=i, min_periods=1).mean())/
                                              (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_semana_1_Dias_Antes']\
                                               .transform(lambda x: x.rolling(window=i, min_periods=1).mean())))-1)


> - Volatilidade
> - Rotura 
> - Supply
> - Percentagem de dias em Stock Borderline
> - Percentagem de dias de Linear Incompleto
> - Percentagem de dias sem vendas
> - Tempo indisponível
> - Vendas perdidas

In [19]:
# Sempre que é pedido abastecimento, fazer com que seja 1
dfFinal["New_Supply"] = np.where((dfFinal["EXPECTED_1_Dias_Antes"]==0) & (dfFinal["EXPECTED"]>0), 1, 0)
dfFinal["Percentagem_Stock_Borderline_1_Dias_Antes"] = np.where(dfFinal["STOCK_1_Dias_Antes"]<0.2*dfFinal["PRES_STOCK_1_Dias_Antes"], 1, 0)
dfFinal["Percentagem_Linear_Incompleto_1_Dias_Antes"] = np.where(dfFinal["STOCK_1_Dias_Antes"]<dfFinal["PRES_STOCK_1_Dias_Antes"], 1, 0)
dfFinal["Sem_Vendas_1_Dias_Antes"] = np.where(dfFinal["SELLOUT_1_Dias_Antes"] == 0, 1, 0)
dfFinal["Dias_Indisponivel_1_Dias_Antes"] = np.where(dfFinal["ROTURA_1_Dias_Antes"]==1, 1, 0)

dfFinal['ROTURA_fds_1_Dias_Antes'] = dfFinal[dfFinal['DATA'].dt.weekday.isin([5,6,0])]["ROTURA_1_Dias_Antes"].copy()
dfFinal['ROTURA_semana_1_Dias_Antes'] = dfFinal[dfFinal['DATA'].dt.weekday.isin([1,2,3,4])]["ROTURA_1_Dias_Antes"].copy()

for i in diasMetHist:
    dfFinal[f"Percentagem_Volatilidade_{i}"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_1_Dias_Antes']\
                                                .transform(lambda x: x.rolling(window=i, min_periods=1).std()) /
                                                dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_1_Dias_Antes']\
                                                .transform(lambda x: x.rolling(window=i, min_periods=1).mean()))
    
    dfFinal[f"Percentagem_Roturas_{i}"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['ROTURA_1_Dias_Antes']\
                                           .transform(lambda x: x.rolling(window=i, min_periods=1).mean()))
    
    dfFinal[f"Percentagem_Roturas_{i}"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['ROTURA_1_Dias_Antes']\
                                           .transform(lambda x: x.rolling(window=i, min_periods=1).mean()))
    
    dfFinal[f"Percentagem_Supply_{i}"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['New_Supply']\
                                          .transform(lambda x: x.rolling(window=i, min_periods=1).mean()))

    dfFinal[f"Percentagem_Dias_Stock_Borderline_{i}"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['Percentagem_Stock_Borderline_1_Dias_Antes']\
                                                         .transform(lambda x: x.rolling(window=i, min_periods=1).mean()))

    dfFinal[f"Percentagem_Dias_Linear_Incompleto_{i}"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['Percentagem_Linear_Incompleto_1_Dias_Antes']\
                                                          .transform(lambda x: x.rolling(window=i, min_periods=1).mean()))
    
    dfFinal[f"Percentagem_Dias_Sem_Vendas_{i}"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['Sem_Vendas_1_Dias_Antes']\
                                                   .transform(lambda x: x.rolling(window=i, min_periods=1).mean()))
    
    dfFinal[f"Percentagem_Dias_Indisponivel_{i}"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['Dias_Indisponivel_1_Dias_Antes']\
                                                     .transform(lambda x: x.rolling(window=i, min_periods=1).mean()))

    dfFinal[f"Vendas_Perdidas_em_{i}_Dias"] = ((dfFinal.groupby(['DESC_ARTIGO', "STORE"])['ROTURA_fds_1_Dias_Antes'].transform(lambda x: x.rolling(window=i, min_periods=1).sum())\
                                                  * dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_fds_1_Dias_Antes'].transform(lambda x: x.rolling(window=i, min_periods=1).mean())) \
                                                  + 
                                               (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['ROTURA_semana_1_Dias_Antes'].transform(lambda x: x.rolling(window=i, min_periods=1).sum()) \
                                                  * dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_semana_1_Dias_Antes'].transform(lambda x: x.rolling(window=i, min_periods=1).mean())))
    

> - Tempo médio inter-supply

In [20]:
dfFinal["InterSupply"] = np.where(dfFinal["EXPECTED"]==0, 1, 0)

groups = (dfFinal['InterSupply'] != dfFinal['InterSupply'].shift()).cumsum()
result = dfFinal.groupby(groups).agg({'DATA': 'first', 'DESC_ARTIGO': 'first', 'STORE': 'first', 'InterSupply': 'sum'}).reset_index(drop=True)
result = result[result['InterSupply'] > 0]

dfFinal = dfFinal.drop(columns=['InterSupply'])

dfFinal = pd.merge(dfFinal, result, how="left", on=["DATA","DESC_ARTIGO", "STORE"])

for i in diasMetHist:
    dfFinal[f"Percentagem_InterSupplyMed_{i}"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['InterSupply'].transform(lambda x: x.rolling(window=i, min_periods=1).mean()))


---

 ### <font color=green> Função Contagem

In [21]:
from datetime import datetime, timedelta


# Define a function to calculate weekday and weekend counts
def contagem5dias(date):
    weekday_count = 0
    weekend_count = 0
    
    for _ in range(5):
        if date.weekday() < 4:  # Monday to Thursday
            weekday_count += 1
        else:  # Friday to Sunday
            weekend_count += 1
        
        date += timedelta(days=1)
    
    return pd.Series({"CONTAGEM_SEMANA": weekday_count, "CONTAGEM_FIMSEMANA": weekend_count})


# Define a function to calculate weekday and weekend counts
def contagem3dias(date):
    weekday_count = 0
    weekend_count = 0
    
    for _ in range(3):
        if date.weekday() < 4:  # Monday to Thursday
            weekday_count += 1
        else:  # Friday to Sunday
            weekend_count += 1
        
        date += timedelta(days=1)
    
    return pd.Series({"CONTAGEM_SEMANA3": weekday_count, "CONTAGEM_FIMSEMANA3": weekend_count})

> - Smart 3 dias

In [22]:
%%time

# Apply the function to each row in the DataFrame
dfFinal[["CONTAGEM_SEMANA3", "CONTAGEM_FIMSEMANA3"]] = dfFinal["DATA"].apply(contagem3dias)
dfFinal['CONTAGEM_FIMSEMANA3'] = dfFinal['CONTAGEM_FIMSEMANA3'].shift(-1)
dfFinal['CONTAGEM_SEMANA3'] = dfFinal['CONTAGEM_SEMANA3'].shift(-1)

dfFinal["Balance_Smart3"] = ((
    (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_fds_1_Dias_Antes']\
    .transform(lambda x: x.rolling(window=30, min_periods=1).mean())* dfFinal['CONTAGEM_FIMSEMANA3'])
    +
    (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_semana_1_Dias_Antes']\
    .transform(lambda x: x.rolling(window=30, min_periods=1).mean())* dfFinal['CONTAGEM_SEMANA3']))
     / dfFinal["STK"])

dfFinal["Balance_Smart3"] = np.where(dfFinal["STK"]<=0, -1, dfFinal["Balance_Smart3"])
dfFinal["Balance_Smart_3_Dias_Antes"] = dfFinal["Balance_Smart3"].shift(2)


CPU times: total: 7.91 s
Wall time: 7.98 s


> - Smart 5 dias antes

In [23]:
%%time

# Apply the function to each row in the DataFrame
dfFinal[["CONTAGEM_SEMANA", "CONTAGEM_FIMSEMANA"]] = dfFinal["DATA"].apply(contagem5dias)
dfFinal['CONTAGEM_FIMSEMANA'] = dfFinal['CONTAGEM_FIMSEMANA'].shift(-1)
dfFinal['CONTAGEM_SEMANA'] = dfFinal['CONTAGEM_SEMANA'].shift(-1)


dfFinal["Balance_Smart"] = ((
    (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_fds_1_Dias_Antes']\
    .transform(lambda x: x.rolling(window=30, min_periods=1).mean())* dfFinal['CONTAGEM_FIMSEMANA'])
    +
    (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_semana_1_Dias_Antes']\
    .transform(lambda x: x.rolling(window=30, min_periods=1).mean())* dfFinal['CONTAGEM_SEMANA']))
     / dfFinal["STK"])

dfFinal["Balance_Smart"] = np.where(dfFinal["STK"]<=0, -1, dfFinal["Balance_Smart"])
dfFinal["Balance_Smart_5_Dias_Antes"] = dfFinal["Balance_Smart"].shift(4)


CPU times: total: 8.19 s
Wall time: 8.32 s


> - Dias para rotura de Stock

In [24]:
dfFinal = dfFinal.copy()
# Dias para a rotura mas com o Sellout médio (móvel) dos últimos 10 dias 
dfFinal["Dias_para_Rotura_Stock"] = dfFinal["STK_1_Dias_Antes"] / (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_fds_1_Dias_Antes']\
    .transform(lambda x: x.rolling(window=30, min_periods=1).mean())* dfFinal['CONTAGEM_FIMSEMANA'])+(dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_semana_1_Dias_Antes']\
    .transform(lambda x: x.rolling(window=30, min_periods=1).mean())* dfFinal['CONTAGEM_SEMANA'])

dfFinal["Dias_para_Rotura_Stock_5_Dias_Antes"] = dfFinal["Dias_para_Rotura_Stock"].shift(4)

> - Dias para rotura de Linear

In [25]:
# Definir a métrica: Preslinear / med(Sellouts 10 dias)
dfFinal['Dias_Duracao_Linear'] = dfFinal["PRES_STOCK"] / (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_fds_1_Dias_Antes']\
    .transform(lambda x: x.rolling(window=30, min_periods=1).mean())* dfFinal['CONTAGEM_FIMSEMANA'])+(dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_semana_1_Dias_Antes']\
    .transform(lambda x: x.rolling(window=30, min_periods=1).mean())* dfFinal['CONTAGEM_SEMANA'])

dfFinal["Dias_Duracao_Linear_5_Dias_Antes"] = dfFinal["Dias_Duracao_Linear"].shift(4)

### Texto

> - Desempenho
> - Volatilidade de Sellout
> - Sensibilidade
> - PS Classificação
> - Risco de Rotura

In [26]:
dfFinal["Desempenho_de_Vendas_60"] = np.where((dfFinal["Percentagem_Dias_Sem_Vendas_60"]>=0) & (dfFinal["Percentagem_Dias_Sem_Vendas_60"]<=0.151), "Vendas com desempenho satisfatório",
                                     np.where((dfFinal["Percentagem_Dias_Sem_Vendas_60"]>0.151) & (dfFinal["Percentagem_Dias_Sem_Vendas_60"]<=0.317), "Vendas com desempenho mediano",
                                     np.where((dfFinal["Percentagem_Dias_Sem_Vendas_60"]>0.317) & (dfFinal["Percentagem_Dias_Sem_Vendas_60"]<=0.433), "Vendas com desempenho insatisfatório",
                                              "Vendas com desempenho muito insatisfatório")))


                                     #np.where((dfFinal["Percentagem_Dias_Sem_Vendas_60"]>0.433) & (dfFinal["Percentagem_Dias_Sem_Vendas_60"]<=1), 

dfFinal["Volatilidade_SELLOUT"] = np.where(dfFinal["Percentagem_Volatilidade_60"] > 1.095, " e com alta volatilidade de sellout.",
                                  np.where(dfFinal["Percentagem_Volatilidade_60"] > 0.91 , " e com média volatilidade de sellout.",
                                  np.where(dfFinal["Percentagem_Volatilidade_60"] > 0.76 , " e com baixa volatilidade de sellout.",
                                                                                           " e com muito baixa volatilidade de sellout.")))


dfFinal["Sensibilidade_Rotura"] = np.where(dfFinal["Percentagem_Roturas_60"]>0.33, " Produto com muito elevada propensão para rotura",
                                 np.where(dfFinal["Percentagem_Roturas_60"]>0.166666, " Produto com elevada propensão para rotura",
                                 np.where(dfFinal["Percentagem_Roturas_60"]>0.017, " Produto com moderada propensão para rotura",
                                 np.where(dfFinal["Percentagem_Roturas_60"]>0, " Produto com baixa propensão para rotura",
                                 " Produto com muito baixa ou nula propensão para rotura"))))

dfFinal["Order_Sensibilidade_Rotura"] = np.where(dfFinal["Percentagem_Roturas_60"]>0.33, 5,
                                 np.where(dfFinal["Percentagem_Roturas_60"]>0.166666, 4,
                                 np.where(dfFinal["Percentagem_Roturas_60"]>0.017, 3,
                                 np.where(dfFinal["Percentagem_Roturas_60"]>0, 2,
                                 1))))

dfFinal["PS_Classificacao_60"] = np.where((dfFinal["Percentagem_Dias_Linear_Incompleto_60"] > 0.51) & (dfFinal["Percentagem_Balance_Raw_Count1_60"]==1) & (dfFinal["Percentagem_Dias_Sem_Vendas_60"]>0.29), " e com PS excessivo.",
                              np.where((dfFinal["Percentagem_Dias_Linear_Incompleto_60"] > 0.31) & (dfFinal["Percentagem_Balance_Raw_Count1_60"]==1) & (dfFinal["Percentagem_Dias_Sem_Vendas_60"]>0.29), " e com PS moderadamente excessivo.",
                              np.where((dfFinal["Percentagem_Dias_Linear_Incompleto_60"] < 0.07) & (dfFinal["Percentagem_Balance_Raw_Count1_60"]<0.9) & (dfFinal["Percentagem_Dias_Sem_Vendas_60"]<0.07), " e com PS escasso.", 
                                                                                                                                                                                                            " e com PS equilibrado.")))

dfFinal["RISCO"] = np.where((dfFinal.Balance_Smart > 2) & (dfFinal.CICLOS_5_Dias_Antes < 0.2) & (dfFinal.Percentagem_Roturas_120 > 5), 1,
                   np.where((dfFinal.Balance_Smart > 1) & (dfFinal.Balance_Smart < 2) & (dfFinal.CICLOS_5_Dias_Antes < 0.6) & (dfFinal.Percentagem_Roturas_120 < 3), 2, 3))

dfFinal["RISCO_ROTURA"] = np.where((dfFinal.Balance_Smart >= 12.5) | (dfFinal.Balance_Smart < 0), " O produto apresenta hoje muito elevada probabilidade de rotura a 5 dias caso não haja fornecimento.", 
                          np.where((dfFinal.Balance_Smart >= 1.1) & (dfFinal.Balance_Smart < 12.5), " O produto apresenta hoje elevada probabilidade de rotura a 5 dias caso não haja fornecimento.",
                          np.where((dfFinal.Balance_Smart >= 0.833) & (dfFinal.Balance_Smart < 1.1), " O produto apresenta hoje média probabilidade de rotura a 5 dias caso não haja fornecimento.",
                          np.where((dfFinal.Balance_Smart >= 0.087) & (dfFinal.Balance_Smart < 0.833), " O produto apresenta hoje baixa probabilidade de rotura a 5 dias caso não haja fornecimento.", 
                                                                                                         " O produto apresenta hoje muito baixa ou nula probabilidade de rotura a 5 dias caso não haja fornecimento."))))         

> - Conclusão

In [27]:
dfFinal["Conclusao1"] = dfFinal["Desempenho_de_Vendas_60"] + dfFinal["Volatilidade_SELLOUT"] 
dfFinal["Conclusao2"] = dfFinal["Sensibilidade_Rotura"] + dfFinal["PS_Classificacao_60"] 
dfFinal["Conclusao3"] = dfFinal["RISCO_ROTURA"]

# Escrever

- Dias certos

In [28]:
dfFinal["ROTURA"] = np.where(dfFinal["ROTURA"]==1, "NÃO", "SIM")
# Ficheiro Dia
dfEscreverDia = dfFinal[dfFinal.DATA == dfFinal.DATA.unique()[-2]].copy()

# Ficheiro Mês
dfEscreverMes = dfFinal[dfFinal.DATA.between(dfFinal.DATA.unique()[-31],dfFinal.DATA.unique()[-2])].copy()

# Ficheiro 180 dias
dfEscrever180 = dfFinal[dfFinal.DATA.between(dfFinal.DATA.unique()[-124],dfFinal.DATA.unique()[-2])].copy()

# Ninjas

In [29]:
# Categorias possíveis
mapping = {1: "Presente com Stock",
           2: "Ausente com Stock",
           3: "Presente sem Stock",
           4: "Ausente sem Stock",
           5: "Presente sem Registo",
           6: "Ausente sem Registo"}

# Definir coluna de sinal
dfFinal["Sinal"] = np.where((dfFinal["STOCK"] > 0) & (dfFinal["NinjaInfo"] == 1), 1,
                     np.where((dfFinal["STOCK"] > 0) & (dfFinal["NinjaInfo"] == 0), 2,
                     np.where((dfFinal["STOCK"] <= 0) & (dfFinal["NinjaInfo"] == 1), 3,
                     np.where((dfFinal["STOCK"] <= 0) & (dfFinal["NinjaInfo"] == 0), 4,
                     np.where((dfFinal["STOCK"].isna()) & (dfFinal["NinjaInfo"] == 1), 5,
                     np.where((dfFinal["STOCK"].isna()) & (dfFinal["NinjaInfo"] == 0), 6,
                     np.nan))))))

# Substituir números pelas expressões escolhidas
dfFinal["Sinal"] = dfFinal["Sinal"].map(mapping)

# Tempo anterior

In [30]:
# Extract year and month from the 'date' column
dfFinal['Ano'] = dfFinal['DATA'].dt.year
dfFinal['Ano_Anterior'] = dfFinal['Ano'] + 1
dfFinal['Mes'] = dfFinal['DATA'].dt.month

# Mês Actual
media_mes_actual = dfFinal.groupby(['Ano', 'Mes'])['SELLOUT'].mean().reset_index()
media_mes_actual.rename(columns={'SELLOUT': 'Media_Sellout_Mensal'}, inplace=True)

# Mês Homólogo
media_mes_homologo = dfFinal.groupby(['Ano_Anterior', 'Mes'])['SELLOUT'].mean().reset_index()
media_mes_homologo.rename(columns={'SELLOUT': 'Media_Sellout_Mensal_Homologo'}, inplace=True)

#drop


# Merge the monthly mean back into the original DataFrame
dfFinal = dfFinal.merge(media_mes_actual, on=['Ano', 'Mes'], how="left")
dfFinal = dfFinal.merge(media_mes_homologo, left_on=['Ano', 'Mes'], right_on=['Ano_Anterior', 'Mes'], how="left")

dfFinal = dfFinal.drop(columns = ["Ano_Anterior_x", "Ano_Anterior_y"])

# Calculate the monthly mean for the previous month
media_mes_anterior = dfFinal.groupby(['Ano', 'Mes'])['SELLOUT'].mean().shift(1).reset_index()
media_mes_anterior.rename(columns={'SELLOUT': 'Media_Sellout_Mensal_Mes_Anterior'}, inplace=True)

dfFinal = dfFinal.merge(media_mes_anterior, on=['Ano', 'Mes'], how="left")

# Clickhouse

In [31]:
# Ficheiro Dia
dfEscreverDia = dfFinal[dfFinal.DATA == dfFinal.DATA.unique()[-2]].copy()

# Ficheiro Mês
dfEscreverMes = dfFinal[dfFinal.DATA.between(dfFinal.DATA.unique()[-31],dfFinal.DATA.unique()[-2])].copy()

# Ficheiro 120 dias
dfEscrever120 = dfFinal[dfFinal.DATA.between(dfFinal.DATA.unique()[-121],dfFinal.DATA.unique()[-2])].copy()

In [32]:
import clickhouse_connect
from clickhouse_driver import Client
from unidecode import unidecode

client = clickhouse_connect.get_client(host='ch.brandsandninjas.com', 
                                       port=443, 
                                       username='chninja', 
                                       password='ku43ueqnB5Q0AYb2C4FsJRTc7qX',
                                       database = 'Delta')

In [33]:
#Estabelecer a base a ser lida

for qualbase in range(1,4):

    if qualbase == 1:
        baseCH = dfEscreverDia.copy()
        tabela = "Delta_Dia" # BaseGoChillDia

    elif qualbase == 2:
        baseCH = dfEscreverMes.copy()
        baseCH.NinjaInfo = baseCH.NinjaInfo.fillna(-1)

        tabela = "Delta_Mes" # BaseGoChillMes

    elif qualbase == 3:
        baseCH = dfEscrever120.copy()
        tabela = "Delta_120dias" # BaseDelta120

    elif qualbase == 4:
        baseCH = dfNinjas.copy()
        tabela = "Delta_Audit" # BaseDeltaNinja

    ## Algumas alterações
    baseCH['DATA']= pd.to_datetime(baseCH['DATA'], format='%Y-%m-%d')  # Passar a Data para datetime

    baseCH.columns = [unidecode(col) for col in baseCH.columns]        # Tirar acentos e afins dos nomes das colunas porque 
                                                                       # o Clickhouse não gosta


    ## Listas para definir os tipos de dados de cada coluna a inserit
    data = ["DATA"]
    texto = [col for col in baseCH.columns if baseCH[col].dtype == 'object']
    inteiros = [col for col in baseCH.columns if baseCH[col].dtype == 'int64' or baseCH[col].dtype == 'int32']
    floats = [col for col in baseCH.columns if baseCH[col].dtype == 'float64']


    ## Mudar inteiros para floats porque senão não pode haver missing values
    for col_name in inteiros:
        baseCH[col_name] = baseCH[col_name].astype(float)

    ## Missing values em strings também estragam tudo
    baseCH[texto] = baseCH[texto].fillna("-")
    baseCH[texto] = baseCH[texto].astype(str)

    #Função que vai fazer a schema
    def schema(lista, tipo):
        result_list = [f"{element} {tipo}" for element in lista]
        return result_list

    # Schema a ser feito
    data1 = schema(data, "Date")
    texto1 = schema(texto, "String")
    inteiros1 = schema(inteiros, "Float64")
    floats1 = schema(floats, "Float64")
    total = tuple(data1 + texto1 + inteiros1 + floats1)
    schema = ', '.join([column.replace("'", "") for column in total])

    # Split the input string by commas
    parts = schema.split(', ')
    # Process each part and wrap the first word in double quotes
    output_parts = []
    for part in parts:
        words = part.split()
        if words:
            first_word = words[0]
            remaining_words = ' '.join(words[1:])
            output_part = f'"{first_word}" {remaining_words}'
            output_parts.append(output_part)
    # Join the modified parts back into a string
    schema = ', '.join(output_parts)

    # Eliminar tabela que possa existir no CH com o mesmo nome
    client.command(f'DROP TABLE IF EXISTS {tabela}')

    # Criar tabela no CH
    client.command(f'''
        CREATE TABLE IF NOT EXISTS {tabela} (
            {schema}
            ) ENGINE = MergeTree
            ORDER BY (DATA)
    ''')


    # Exportar os dados para o clickhouse
    client.insert_df(tabela, baseCH, column_names=baseCH.columns.tolist())

# Análises

##  Estudos